# **Question 3**

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load and preprocess MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshape and normalize input images
X_train = X_train.reshape(-1, 28*28).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28*28).astype('float32') / 255.0

# One-hot encode target labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the neural network architecture
model = models.Sequential([
    layers.Dense(256, activation='relu', input_shape=(28*28,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_data=(X_val, y_val))

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

Epoch 1/10
375/375 [==============================] - 7s 7ms/step - loss: 0.3200 - accuracy: 0.9066 - val_loss: 0.1447 - val_accuracy: 0.9555
Epoch 2/10
375/375 [==============================] - 1s 4ms/step - loss: 0.1199 - accuracy: 0.9636 - val_loss: 0.1052 - val_accuracy: 0.9688
Epoch 3/10
375/375 [==============================] - 1s 4ms/step - loss: 0.0771 - accuracy: 0.9762 - val_loss: 0.0931 - val_accuracy: 0.9711
Epoch 4/10
375/375 [==============================] - 1s 3ms/step - loss: 0.0557 - accuracy: 0.9827 - val_loss: 0.0848 - val_accuracy: 0.9738
Epoch 5/10
375/375 [==============================] - 1s 3ms/step - loss: 0.0396 - accuracy: 0.9876 - val_loss: 0.0822 - val_accuracy: 0.9759
Epoch 6/10
375/375 [==============================] - 1s 3ms/step - loss: 0.0322 - accuracy: 0.9896 - val_loss: 0.0932 - val_accuracy: 0.9730
Epoch 7/10
375/375 [==============================] - 1s 4ms/step - loss: 0.0264 - accuracy: 0.9912 - val_loss: 0.0789 - val_accuracy: 0.9793
Epoch 

# **Question** **4**

# a) **Alexnet**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Define transforms for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the SVHN dataset
full_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform)

# Split the dataset into training and testing sets
train_indices, test_indices = train_test_split(range(len(full_dataset)), test_size=0.75, random_state=42)

# Define data loaders for the reduced dataset
train_loader = DataLoader(full_dataset, batch_size=64, sampler=SubsetRandomSampler(train_indices))
test_loader = DataLoader(full_dataset, batch_size=64, sampler=SubsetRandomSampler(test_indices))

# Define the AlexNet model
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Initialize the model, loss function, and optimizer
model = AlexNet(num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
for epoch in range(5):  # Number of epochs
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

# Evaluation
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print('Accuracy on test set: %.2f%%' % (accuracy * 100))


100%|██████████| 182040794/182040794 [00:08<00:00, 21289533.53it/s]


[1,   100] loss: 2.269
[1,   200] loss: 2.266
[2,   100] loss: 2.270
[2,   200] loss: 2.271
[3,   100] loss: 2.262
[3,   200] loss: 2.272
[4,   100] loss: 2.260
[4,   200] loss: 2.270
[5,   100] loss: 2.269
[5,   200] loss: 2.270
Finished Training
Accuracy on test set: 18.79%


# b)**LeNet**-5

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Define transforms for the dataset
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the SVHN dataset
full_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform)

# Split the dataset into training and testing sets
train_indices, test_indices = train_test_split(range(len(full_dataset)), test_size=0.75, random_state=42)

# Define data loaders for the reduced dataset
train_loader = DataLoader(full_dataset, batch_size=64, sampler=SubsetRandomSampler(train_indices))
test_loader = DataLoader(full_dataset, batch_size=64, sampler=SubsetRandomSampler(test_indices))

# Define the LeNet-5 model
class LeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss function, and optimizer
model = LeNet5(num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
for epoch in range(5):  # Number of epochs
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

# Evaluation
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print('Accuracy on test set: %.2f%%' % (accuracy * 100))


Using downloaded and verified file: ./data/train_32x32.mat
[1,   100] loss: 2.241
[1,   200] loss: 1.962
[2,   100] loss: 0.998
[2,   200] loss: 0.874
[3,   100] loss: 0.712
[3,   200] loss: 0.632
[4,   100] loss: 0.556
[4,   200] loss: 0.514
[5,   100] loss: 0.471
[5,   200] loss: 0.463
Finished Training
Accuracy on test set: 84.81%


# c) ResNet-**18**

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import torchvision.models as models

# Define transforms for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the SVHN dataset
full_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform)

# Split the dataset into training and testing sets
train_indices, test_indices = train_test_split(range(len(full_dataset)), test_size=0.75, random_state=42)

# Define data loaders for the reduced dataset
train_loader = DataLoader(full_dataset, batch_size=64, sampler=SubsetRandomSampler(train_indices))
test_loader = DataLoader(full_dataset, batch_size=64, sampler=SubsetRandomSampler(test_indices))

# Define the ResNet-18 model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

# Initialize the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop for 5 epochs
for epoch in range(5):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if i % 100 == 99:    # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f, train_accuracy: %.2f %%' %
                  (epoch + 1, i + 1, running_loss / 100, 100 * correct / total))
            running_loss = 0.0

print('Finished Training')

# Evaluation
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print('Accuracy on test set: %.2f%%' % (accuracy * 100))


Using downloaded and verified file: ./data/train_32x32.mat


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 160MB/s]


[1,   100] loss: 0.709, train_accuracy: 77.62 %
[1,   200] loss: 0.404, train_accuracy: 82.52 %
[2,   100] loss: 0.287, train_accuracy: 91.19 %
[2,   200] loss: 0.323, train_accuracy: 90.71 %
[3,   100] loss: 0.242, train_accuracy: 92.98 %
[3,   200] loss: 0.212, train_accuracy: 93.39 %
[4,   100] loss: 0.164, train_accuracy: 94.73 %
[4,   200] loss: 0.206, train_accuracy: 94.30 %
[5,   100] loss: 0.147, train_accuracy: 95.78 %
[5,   200] loss: 0.150, train_accuracy: 95.68 %
Finished Training
Accuracy on test set: 90.89%


## ***d) VGG-16 ***

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import torchvision.models as models

# Define transforms for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the SVHN dataset
full_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform)

# Split the dataset into training and testing sets
train_indices, test_indices = train_test_split(range(len(full_dataset)), test_size=0.75, random_state=42)

# Define data loaders for the reduced dataset
train_loader = DataLoader(full_dataset, batch_size=64, sampler=SubsetRandomSampler(train_indices))
test_loader = DataLoader(full_dataset, batch_size=64, sampler=SubsetRandomSampler(test_indices))

# Define the VGG-16 model
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 10)

# Initialize the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop for 5 epochs
for epoch in range(5):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if i % 100 == 99:    # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f, train_accuracy: %.2f %%' %
                  (epoch + 1, i + 1, running_loss / 100, 100 * correct / total))
            running_loss = 0.0

print('Finished Training')

# Evaluation
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print('Accuracy on test set: %.2f%%' % (accuracy * 100))


Using downloaded and verified file: ./data/train_32x32.mat


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 76.3MB/s]


[1,   100] loss: 2.262, train_accuracy: 18.94 %
[1,   200] loss: 2.243, train_accuracy: 19.23 %
[2,   100] loss: 2.232, train_accuracy: 19.64 %
[2,   200] loss: 2.245, train_accuracy: 19.27 %
[3,   100] loss: 40.645, train_accuracy: 18.30 %
[3,   200] loss: 2.249, train_accuracy: 18.48 %
[4,   100] loss: 2.246, train_accuracy: 18.53 %
[4,   200] loss: 2.233, train_accuracy: 19.42 %
[5,   100] loss: 2.239, train_accuracy: 19.00 %
[5,   200] loss: 2.241, train_accuracy: 19.02 %
Finished Training
Accuracy on test set: 18.79%
